In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as mpl
import scipy as sp
import pandas as pd

from mayavi import mlab

sys.path.append("./code/bin/")
import ZernikeFunc as ZF
import SurfaceFunc as SF

In [ ]:
Npixel = 25   # the plane edge in pixels...

Rs     = 6    # the radius of the sphere that includes the patch..

Dpp = 1       # the distance between points of the same patch (needed to remove islands)

ZOrder = 20   # the Zernike expansion order..

In [ ]:
### To load a new surface:
surf_name_a =  "4bs2_RRM2.dms"

surf_a_ = pd.read_csv(surf_name_a)
    
l_a = len(surf_a_["x"])
print("Npoints", l_a)

surf_a = np.zeros((l_a, 6))
surf_a[:,:] = surf_a_[["x", "y", "z", "Nx", "Ny", "Nz"]]



### To inizialize the Surface class:
surf_a_obj = SF.Surface(surf_a[:,:], patch_num = 0, r0 = Rs, theta_max = 45)

In [ ]:
## To isolate one patch of the surface:
## Example, the patch around point 5000..
patch, _ = surf_a_obj.BuildPatch(point_pos=5000, Dmin=.5)

patch2, _ = surf_a_obj.BuildPatch(point_pos=19841, Dmin=.5)

In [ ]:
# plotting surface + patch
res1, c = SF.ConcatenateFigPlots([surf_a_obj.surface[:,:3],patch[:,:3]])
SF.Plot3DPoints(res1[:,0], res1[:,1], res1[:,2], c, 0.3)

In [ ]:
# plotting surface + patch2
res1, c = SF.ConcatenateFigPlots([surf_a_obj.surface[:,:3],patch2[:,:3]])
SF.Plot3DPoints(res1[:,0], res1[:,1], res1[:,2], c, 0.3)

In [ ]:
## To rotate a patch:    
rot_patch, rot_patch_nv = surf_a_obj.PatchReorientNew(patch, 1)

In [ ]:
z = surf_a_obj.FindOrigin(rot_patch)
plane, weigths, dist_plane, thetas = surf_a_obj.CreatePlane(patch=rot_patch, z_c=z , Np=Npixel)
            
new_plane = surf_a_obj.FillTheGap_everywhere(plane_=plane)
## enlarging plane..
new_plane_ =  surf_a_obj.EnlargePixels(new_plane)


# plotting res..
fig, ax = mpl.subplots(1,2, dpi = 150)
ax[0].imshow(plane)
ax[1].imshow(new_plane_)
ax[0].axis('off')
ax[1].axis('off')
ax[0].set_title("original")
ax[1].set_title("processed")
mpl.show()

In [ ]:
zernike_env = ZF.Zernike2d(new_plane_)

In [ ]:
coeff = zernike_env.ZernikeDecomposition(order=ZOrder)

In [ ]:
coeff_inv = np.absolute(coeff)

In [ ]:
fig, ax = mpl.subplots(nrows=1, ncols=1, figsize=(5,4), facecolor="white", dpi=150)
data = plane
min_data = np.amin(data)
max_data = np.amax(data)
ax.set_title("Original", fontsize="12")
ax.set_xlabel("x", fontsize="8")
ax.set_ylabel("y", fontsize="8")
ax.tick_params(axis="both", width ="0.50", color="black", labelsize="8")
im = ax.pcolormesh(data, cmap="Greens")
for side in ax.spines.keys():  # 'top', 'bottom', 'left', 'right'
    ax.spines[side].set_linewidth(0.50)
    ax.spines[side].set_color("black")
cb = mpl.colorbar(im, ax=ax, ticks=[min_data, 0.25*max_data, 0.5*max_data, 0.75*max_data, max_data])
cb.ax.tick_params(axis="both", width ="0.30", color="black", labelsize="8")
for side in cb.ax.spines.keys():  # 'top', 'bottom', 'left', 'right'
    cb.ax.spines[side].set_linewidth(0.50)
    cb.ax.spines[side].set_color("black")

#mpl.savefig("original_5000.png")

In [ ]:
fig, ax = mpl.subplots(nrows=1, ncols=1, figsize=(5,4), facecolor="white", dpi=150)
data = new_plane_
min_data = np.amin(data)
max_data = np.amax(data)
ax.set_title("Processed", fontsize="12")
ax.set_xlabel("x", fontsize="8")
ax.set_ylabel("y", fontsize="8")
ax.tick_params(axis="both", width ="0.50", color="black", labelsize="8")
im = ax.pcolormesh(data, cmap="Greens")
for side in ax.spines.keys():  # 'top', 'bottom', 'left', 'right'
    ax.spines[side].set_linewidth(0.50)
    ax.spines[side].set_color("black")
cb = mpl.colorbar(im, ax=ax, ticks=[min_data, 0.25*max_data, 0.5*max_data, 0.75*max_data, max_data])
cb.ax.tick_params(axis="both", width ="0.30", color="black", labelsize="8")
for side in cb.ax.spines.keys():  # 'top', 'bottom', 'left', 'right'
    cb.ax.spines[side].set_linewidth(0.50)
    cb.ax.spines[side].set_color("black")

#mpl.savefig("processed_5000.png")

In [ ]:
fig, ax = mpl.subplots(nrows=1, ncols=1, figsize=(6,4), facecolor="white", dpi=150)
ax.set_xlabel("Index (n,m)")
ax.set_ylabel("Zernike Invariant $|c_{nm}|$")
ax.tick_params(axis="both", width ="0.50", color="black", labelsize="8")
for side in ax.spines.keys():  # 'top', 'bottom', 'left', 'right'
    ax.spines[side].set_linewidth(0.50)
    ax.spines[side].set_color("black")
ax.plot(coeff_inv, lw="1")

#mpl.savefig("invariant_5000.png")